In [174]:
%pip install ollama

Note: you may need to restart the kernel to use updated packages.


In [175]:
"""This script evaluates an LLM prompt for processing text so that it can be used for the wttr.in API"""
 
from ollama import Client

LLM_MODEL: str = "gemma3:4b"    # Optional, change this to be the model you want
client: Client = Client(
  host='http://127.0.0.1:11434' # Optional, change this to be the URL of your LLM
)

In [176]:
# TODO: define  llm_parse_for_wttr()
def llm_parse_for_wttr(input_str) -> str:
    # Give the LLM a post
    # Strip the reponse for the result
    llmResponse = client.chat(
        model=LLM_MODEL,
        messages=[
            {
                "role": "user",
                "content": f"""You are an expert at extracting location information from text and formatting it for use with the *wttr.in* service. Your goal is to accurately identify locations and format them according to specific rules.
 
**Instructions:**
 
Given a sentence about the weather, extract the location mentioned and format it as follows:
 
1. **Lowercase:** Convert all locations to lowercase.
2. **Airports:** If the location is an airport, use its 3-letter IATA code.  *Always prioritize using the 3-letter code if "airport" or similar terms are present.*
3. **Multi-word Locations (Non-Airport):** If the location contains spaces and is *not* an airport, replace each space with a '+'.
4. **Landmarks/Regions:** If the location is a landmark, historical site, region, or any place that is *not* a city or airport, prepend a '~' to the output.  This includes places like "Eiffel Tower", "Machu Picchu", and "Golden Gate Bridge".
 
**Examples:**
 
Input: What is the weather in Denver?
Output: denver
 
Input: How warm is it at Ronald Reagan International Airport?
Output: dca
 
Input: How cold is it going to be in New Mexico?
Output: new+mexico
 
Input: Give me the weather of DCA.
Output: dca
 
Input: What is the weather at the Statue of Liberty?
Output: ~statue+of+liberty
 
Input: Is it going to snow at the Colosseum?
Output: ~colosseum
 
Input: What is the weather at Machu Picchu?
Output: ~machu+picchu
 
Input: Weather in Denver airport
Output: den
 
Input: Give me the weather at colorado springs airport
Output: cos
 
**Now, apply these rules to the following sentence:**
 
                **{input_str}**""",
            },
        ],
    )

    return llmResponse.message.content

In [177]:
# Test cases
test_cases = [  # TODO: Replace these test cases with ones for wttr.in
    {"input": "What's the weather in Rio Rancho?", "expected": "rio+rancho"},
    {"input": "What's the weather in Rio De Janeiro?", "expected": "rio+de+janeiro"},
    {"input": "What is the weather of the Eiffel Tower?", "expected": "~eiffel+tower"},
    {"input": "What is the weather of the London?", "expected": "london"},
    {"input": "What is the weather at the Taj Mahal?", "expected": "~taj+mahal"},
    {"input": "Give me the weather of Machu Picchu?", "expected": "~machu+picchu"},
    {"input": "Weather at colorado springs airport", "expected": "cos"},
    {"input": "Weather in Denver", "expected": "denver"},
    {"input": "Weather in denver airport", "expected": "den"},
    {"input": "How cold will it be in Miami", "expected": "miami"},
    {"input": "What is the temperature in Houston", "expected": "houston"},
    {"input": "What's the weather like at the golden Gate bridge?", "expected": "~golden+gate+bridge"},
    {"input": "Give me the weather at colorado springs airport", "expected": "cos"},
]

In [178]:
def run_tests(test_cases: list[dict[str, str]]):
    """run_tests iterates through a list of dictionaries,
    runs them against an LLM, and reports the results."""
    num_passed = 0

    for i, test in enumerate(test_cases, 1):
        raw_input = test["input"]
        expected_output = test["expected"]

        print(f"\nTest {i}: {raw_input}")
        try:
            result = llm_parse_for_wttr(raw_input).strip()
            expected = expected_output.strip()

            print("LLM Output  :", result)
            print("Expected    :", expected)

            if result == expected:
                print("✅ PASS")
                num_passed += 1
            else:
                print("❌ FAIL")

        except Exception as e:
            print("💥 ERROR:", e)

    print(f"\nSummary: {num_passed} / {len(test_cases)} tests passed.")

In [179]:
#Run Test Cases
run_tests(test_cases=test_cases)


Test 1: What's the weather in Rio Rancho?
LLM Output  : rio+rancho
Expected    : rio+rancho
✅ PASS

Test 2: What's the weather in Rio De Janeiro?
LLM Output  : rio+de+janeiro
Expected    : rio+de+janeiro
✅ PASS

Test 3: What is the weather of the Eiffel Tower?
LLM Output  : ~eiffel+tower
Expected    : ~eiffel+tower
✅ PASS

Test 4: What is the weather of the London?
LLM Output  : london
Expected    : london
✅ PASS

Test 5: What is the weather at the Taj Mahal?
LLM Output  : ~taj+mahal
Expected    : ~taj+mahal
✅ PASS

Test 6: Give me the weather of Machu Picchu?
LLM Output  : ~machu+picchu
Expected    : ~machu+picchu
✅ PASS

Test 7: Weather at colorado springs airport
LLM Output  : cos
Expected    : cos
✅ PASS

Test 8: Weather in Denver
LLM Output  : denver
Expected    : denver
✅ PASS

Test 9: Weather in denver airport
LLM Output  : den
Expected    : den
✅ PASS

Test 10: How cold will it be in Miami
LLM Output  : mia
Expected    : miami
❌ FAIL

Test 11: What is the temperature in Housto